In [39]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
model = pybamm.lithium_ion.DFN(options={"calculate discharge energy":"true"})
for event in model.events: 
    if event.name == "Zero electrolyte concentration cut-off":
        model.events.remove(event)
chem = pybamm.parameter_sets.Chen2020 #Cell Chemistry
#chem 

In [ ]:
Parameter_values = pybamm.ParameterValues(chemistry=chem)
df = pd.DataFrame()
# Set number of var_pts for the mesh
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p:400, var.r_n: 150, var.r_p: 20}
C_rates = (1,2)
sims = []
Energy = []
Power = []
Factor = []
param_name = "Initial concentration in electrolyte [mol.m-3]"
param_list = (0.5,1)
for factor in param_list: 
     Factor.append(factor)
     sims = []
     Parameter_values = pybamm.ParameterValues(chemistry=chem)
     Parameter_values[param_name] *= factor
     for C_rate in C_rates:
     #Parameter_values["Current function [A]"] = 5 * C_rate
     #sim = pybamm.Simulation(model, parameter_values=Parameter_values)
     #sim.solve([0, 4000 / C_rate])
     #sims.append(sim)
          if C_rate < 2.1:
               t_eval = np.linspace(0, 4000 / C_rate, 500)
               dt_max = None
          elif C_rate < 4:
               t_eval = np.linspace(0, 3500 / C_rate, 1000)
               dt_max = 0.0001
          elif C_rate < 8:
               t_eval = np.linspace(0, 3000 / C_rate, 500)
               dt_max = 0.0001
          else:
               t_eval = np.linspace(0, 60, 500)
               dt_max = 0.0001
          solver = pybamm.CasadiSolver(mode="safe", dt_max=dt_max, max_step_decrease_count=15)
          power = C_rate*15 
          experiment = pybamm.Experiment([f"Discharge at {power} W until 2.5 V"], period="40 seconds")
          sim = pybamm.Simulation(model, parameter_values=Parameter_values, var_pts=var_pts, solver=solver, C_rate=C_rate, experiment=experiment)
          sim.solve()
          sims.append(sim)
          print(sim.solution.termination)
          Energy.append([sol.solution["Discharge energy [W.h]"].data[-1] for sol in sims])
          Power.append([sol.solution["Power [W]"].data[-1] for sol in sims])
     zipped = list(zip(Energy, Power, Factor))
     df = pd.DataFrame(zipped, columns=['Energy', 'Power', 'Factor'])
     df.to_csv()
    

In [45]:
df

,Energy,Power,Factor
0,[17.163994561477793],[15.000007187593594],0.5
1,"[17.163994561477793, 11.981139855125631]","[15.000007187593594, 30.000000453347123]",1.0


In [46]:
Energy

[[17.163994561477793],
 [17.163994561477793, 11.981139855125631],
 [17.350405925760214],
 [17.350405925760214, 15.479696756472308]]

In [47]:
Power

[[15.000007187593594],
 [15.000007187593594, 30.000000453347123],
 [15.000001913923242],
 [15.000001913923242, 30.000002081622352]]

In [48]:
Factor

[0.5, 1]